In [1]:
# Instalar PySpark y Spark NLP
! pip install -q pyspark==3.3.0 spark-nlp==4.2.8

# Instalar visualizador de Spark NLP Display
! pip install --upgrade -q spark-nlp-display

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.3/281.3 MB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 453.8/453.8 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.6/95.6 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 31.3 MB/s eta 0:00:00


In [2]:
import json
import pandas as pd
import numpy as np

import sparknlp
import pyspark.sql.functions as F

from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
from sparknlp.annotator import *
from sparknlp.base import *
from sparknlp.pretrained import PretrainedPipeline
from pyspark.sql.types import StringType, IntegerType

In [3]:
# verificacion de versiones

spark = sparknlp.start()

print("Spark NLP version", sparknlp.version())
print("Apache Spark version:", spark.version)

spark

Spark NLP version 4.2.8
Apache Spark version: 3.3.0


In [4]:
# Modelos disponibles: wikiner_6B_100, wikiner_6B_300, wikiner_840B_300

MODEL_NAME = "wikiner_840B_300"

In [6]:
# Primer texto a transformar
text_list = ["""William Henry Gates III (nacido el 28 de octubre de 1955) es un magnate de los negocios, desarrollador de software, inversor y filántropo estadounidense. Es mejor conocido como el cofundador de Microsoft Corporation. Durante su carrera en Microsoft, Gates ocupó los cargos de presidente, director ejecutivo (CEO), presidente y arquitecto de software en jefe, y también fue el mayor accionista individual hasta mayo de 2014. Es uno de los empresarios y pioneros más conocidos de revolución de la microcomputadora de los años setenta y ochenta. Nacido y criado en Seattle, Washington, Gates cofundó Microsoft con su amigo de la infancia Paul Allen en 1975, en Albuquerque, Nuevo México; se convirtió en la compañía de software de computadora personal más grande del mundo. Gates dirigió la compañía como presidente y CEO hasta que dejó el cargo de CEO en enero de 2000, pero siguió siendo presidente y se convirtió en el arquitecto jefe de software. A fines de la década de 1990, Gates había sido criticado por sus tácticas comerciales, que se han considerado anticompetitivas. Esta opinión ha sido confirmada por numerosas sentencias judiciales. En junio de 2006, Gates anunció que haría la transición a un puesto de medio tiempo en Microsoft y trabajaría a tiempo completo en la Fundación Bill y Melinda Gates, la fundación caritativa privada que él y su esposa, Melinda Gates, establecieron en 2000. [ 9] Poco a poco transfirió sus deberes a Ray Ozzie y Craig Mundie. Renunció como presidente de Microsoft en febrero de 2014 y asumió un nuevo cargo como asesor tecnológico para apoyar al recién nombrado CEO Satya Nadella.""",
             """La Mona Lisa es una pintura al óleo del siglo XVI creada por Leonardo. Se celebra en el Louvre de París."""]

In [7]:
text_list

['William Henry Gates III (nacido el 28 de octubre de 1955) es un magnate de los negocios, desarrollador de software, inversor y filántropo estadounidense. Es mejor conocido como el cofundador de Microsoft Corporation. Durante su carrera en Microsoft, Gates ocupó los cargos de presidente, director ejecutivo (CEO), presidente y arquitecto de software en jefe, y también fue el mayor accionista individual hasta mayo de 2014. Es uno de los empresarios y pioneros más conocidos de revolución de la microcomputadora de los años setenta y ochenta. Nacido y criado en Seattle, Washington, Gates cofundó Microsoft con su amigo de la infancia Paul Allen en 1975, en Albuquerque, Nuevo México; se convirtió en la compañía de software de computadora personal más grande del mundo. Gates dirigió la compañía como presidente y CEO hasta que dejó el cargo de CEO en enero de 2000, pero siguió siendo presidente y se convirtió en el arquitecto jefe de software. A fines de la década de 1990, Gates había sido cri

In [13]:
text_list1 = ['''Una juez de garantías aplicó este domingo la medida cautelar de reporte los días lunes y jueves e imputó cargos por posesión de drogas a un español, un colombiano, un venezolano y a un panameño, quienes fueron aprehendidos con 28 ampolletas de fentanilo. Tras un operativo realizado por la Policía el viernes, se informó de la detención de estas personas con el fentanilo y cierta cantidad de marihuana en el corregimiento de Pueblo Nuevo. La juez de garantías, Jazmín Jaén, legalizó la aprehensión y acogió la imputación presentada por la fiscal de drogas, Delia Burgos, por posesión de fentanilo y marihuana.
La detención de los sospechosos se produjo durante una operativo policial realizado en calle sexta Pueblo Nuevo, a raíz de una disputa entre pandillas que obligó a la intervención de la Policía. Durante una diligencia de allanamiento, realizada a una de las residencias, se ubicó un maletín con varias cajas de medicamentos, entre ellos 28 viales de fentanilo. Se informó que el las viales de fentanilo fueron enviadas al Instituto de Medicinal Legal y Ciencias Forenses para su análisis y determinar si corresponden a lote que fue sustraído de la Caja de Seguro Social (CSS) en diciembre pasado.Se informó que la fiscal Burgos apeló las medidas cautelares dispuestas por la juez de garantías, por lo que se fijó como fecha de la audiencia de apelación el próximo 2 de junio a las 8.30 a.m. ante el Tribunal Superior de Apelaciones.
Se trata de uno de los primeros decomisos de esta droga realizado por las autoridades. El Ministerio Público sigue con las investigaciones tras la desaparición de cerca de 20 mil ampolletas de fentanilo, de los depósitos de la Caja de Seguro Social.''']

Creación del Pipeline de NLP con Spark

In [8]:
document_assembler = DocumentAssembler() \
    .setInputCol('text') \
    .setOutputCol('document')

tokenizer = Tokenizer() \
    .setInputCols(['document']) \
    .setOutputCol('token')

# The wikiner_840B_300 is trained with glove_840B_300, so the embeddings in the
# pipeline should match
if MODEL_NAME == "wikiner_840B_300":
    embeddings = WordEmbeddingsModel.pretrained('glove_840B_300', lang='xx') \
        .setInputCols(["document", 'token']) \
        .setOutputCol("embeddings")

ner_model = NerDLModel.pretrained(MODEL_NAME, 'es') \
    .setInputCols(['document', 'token', 'embeddings']) \
    .setOutputCol('ner')

ner_converter = NerConverter() \
    .setInputCols(['document', 'token', 'ner']) \
    .setOutputCol('ner_chunk')

nlp_pipeline = Pipeline(
    stages=[
        document_assembler, 
        tokenizer,
        embeddings,
        ner_model,
        ner_converter])

glove_840B_300 download started this may take some time.
Approximate size to download 2.3 GB
[OK!]
wikiner_840B_300 download started this may take some time.
Approximate size to download 14.2 MB
[OK!]


Ejecución del pipeline

In [9]:
df = spark.createDataFrame(text_list, StringType()).toDF("text")


In [10]:
df

DataFrame[text: string]

In [11]:
result = nlp_pipeline.fit(df).transform(df)

Visualizar resultados de las entidades principales (Key Entities)

In [12]:
from sparknlp_display import NerVisualizer

NerVisualizer().display(
    result = result.collect()[0],
    label_col = 'ner_chunk',
    document_col = 'document'
)

In [14]:
df1 = spark.createDataFrame(text_list1, StringType()).toDF("text")

In [16]:
result1 = nlp_pipeline.fit(df1).transform(df1)

In [18]:
NerVisualizer().display(
    result = result1.collect()[0],
    label_col = 'ner_chunk',
    document_col = 'document'
)